In [2]:
# ============================================
# 🔥 RÉENTRAÎNEMENT DU MODÈLE FINAL
# ============================================

print("="*70)
print("🔥 RÉENTRAÎNEMENT DU MODÈLE LOGISTICREGRESSION")
print("="*70)

import pandas as pd
import numpy as np
import joblib
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack, csr_matrix
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 1. CHARGEMENT DES DONNÉES
# ============================================

print("\n📦 1. CHARGEMENT DES DONNÉES COMPLÈTES")
print("-"*70)

df = pd.read_csv('../data/spam.csv', encoding='latin-1')
df = df[['v1', 'v2']]
df.columns = ['label', 'message']

print(f"✅ Données chargées: {len(df)} messages")
print(f"   • HAM: {sum(df['label'] == 'ham')} messages")
print(f"   • SPAM: {sum(df['label'] == 'spam')} messages")

# Convertir les labels
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

# ============================================
# 2. NETTOYAGE DU TEXTE
# ============================================

print("\n🧹 2. NETTOYAGE DU TEXTE")
print("-"*70)

def clean_text(text):
    """Nettoie le texte"""
    text = str(text).lower()
    
    # Supprimer URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Supprimer emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # Supprimer numéros de téléphone
    text = re.sub(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]', '', text)
    
    # Supprimer ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Supprimer chiffres
    text = re.sub(r'\d+', '', text)
    
    # Supprimer caractères spéciaux
    text = re.sub(r'[^\w\s]', '', text)
    
    return text

df['clean_message'] = df['message'].apply(clean_text)
print("✅ Texte nettoyé")

# ============================================
# 3. VECTORISATION TF-IDF
# ============================================

print("\n🔤 3. VECTORISATION TF-IDF")
print("-"*70)

# Créer le vectorizer
vectorizer = TfidfVectorizer(
    max_features=1000,
    min_df=5,
    max_df=0.7,
    stop_words='english',
    ngram_range=(1, 2)
)

# Ajuster et transformer
X_tfidf = vectorizer.fit_transform(df['clean_message'])
print(f"✅ TF-IDF créé: {X_tfidf.shape[1]} features")

# ============================================
# 4. FEATURES NUMÉRIQUES (16 features)
# ============================================

print("\n🔢 4. EXTRACTION DES FEATURES NUMÉRIQUES")
print("-"*70)

def extract_numeric_features(text):
    """Extrait 16 features numériques"""
    # 1. Longueur
    char_count = len(text)
    word_count = len(text.split())
    avg_word_length = char_count / max(word_count, 1)
    
    # 2. Mots suspects de spam
    spam_keywords = [
        'free', 'win', 'winner', 'won', 'prize', 'cash', 
        'urgent', 'congratulations', 'claim', 'offer',
        'money', 'guaranteed', 'risk', 'limited'
    ]
    
    keyword_features = []
    for keyword in spam_keywords[:8]:  # Prendre 8 mots-clés
        keyword_features.append(1 if keyword in text.lower() else 0)
    
    # 3. Ponctuation et majuscules
    exclamation_count = text.count('!')
    question_count = text.count('?')
    upper_case_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
    
    # 4. Flags
    is_long_message = 1 if char_count > 100 else 0
    has_punctuation = 1 if ('!' in text or '?' in text) else 0
    
    # Total: 16 features
    features = [
        char_count,
        word_count,
        avg_word_length,
        *keyword_features,  # 8 features
        exclamation_count,
        question_count,
        upper_case_ratio,
        is_long_message,
        has_punctuation
    ]
    
    return np.array(features, dtype=np.float32)

# Extraire pour tous les messages
numeric_features_list = []
for text in df['message']:
    numeric_features = extract_numeric_features(text)
    numeric_features_list.append(numeric_features)

X_numeric = np.array(numeric_features_list)
X_numeric_sparse = csr_matrix(X_numeric)

print(f"✅ Features numériques extraites: {X_numeric.shape[1]} features")

# ============================================
# 5. COMBINAISON DES FEATURES
# ============================================

print("\n🔗 5. COMBINAISON DES FEATURES")
print("-"*70)

# Combiner TF-IDF (1000) + Numériques (16) = 1016
X_combined = hstack([X_tfidf, X_numeric_sparse])
y = df['label_num'].values

print(f"✅ Features combinées: {X_combined.shape[1]} features total")
print(f"   • TF-IDF: {X_tfidf.shape[1]}")
print(f"   • Numériques: {X_numeric.shape[1]}")
print(f"   • Cibles: {len(y)}")

# ============================================
# 6. SPLIT TRAIN/TEST
# ============================================

print("\n📊 6. SPLIT DES DONNÉES")
print("-"*70)

X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print(f"✅ Split terminé:")
print(f"   • Train: {X_train.shape[0]} échantillons")
print(f"   • Test: {X_test.shape[0]} échantillons")

# ============================================
# 7. ENTRAÎNEMENT DU MODÈLE
# ============================================

print("\n🤖 7. ENTRAÎNEMENT DU MODÈLE LOGISTICREGRESSION")
print("-"*70)

# Créer et entraîner le modèle
model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced',  # Important car données déséquilibrées
    C=1.0,
    solver='liblinear'
)

model.fit(X_train, y_train)
print("✅ Modèle entraîné")

# ============================================
# 8. ÉVALUATION
# ============================================

print("\n📈 8. ÉVALUATION DU MODÈLE")
print("-"*70)

# Prédictions
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

# Métriques
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"📊 PERFORMANCES:")
print(f"   • Accuracy (train): {train_accuracy:.2%}")
print(f"   • Accuracy (test):  {test_accuracy:.2%}")
print(f"   • Precision:        {precision:.2%}")
print(f"   • Recall:           {recall:.2%}")
print(f"   • F1-Score:         {f1:.2%}")

# Rapport détaillé
print(f"\n📋 RAPPORT DE CLASSIFICATION:")
print(classification_report(y_test, y_pred, target_names=['HAM', 'SPAM']))

# ============================================
# 9. SAUVEGARDE DES MODÈLES
# ============================================

print("\n💾 9. SAUVEGARDE DES MODÈLES")
print("-"*70)

import os
os.makedirs('../models', exist_ok=True)

# Sauvegarder le modèle
joblib.dump(model, '../models/logistic_regression_model_final.joblib')
print("✅ Modèle sauvegardé: models/logistic_regression_model_final.joblib")

# Sauvegarder le vectorizer
joblib.dump(vectorizer, '../models/tfidf_vectorizer_final.joblib')
print("✅ Vectorizer sauvegardé: models/tfidf_vectorizer_final.joblib")

# Sauvegarder l'encoder de labels
label_encoder = {'ham': 0, 'spam': 1}
joblib.dump(label_encoder, '../models/label_encoder_final.joblib')
print("✅ Label encoder sauvegardé: models/label_encoder_final.joblib")

# ============================================
# 10. TESTS DE PRÉDICTION
# ============================================

print("\n🧪 10. TESTS DE PRÉDICTION")
print("-"*70)

test_messages = [
    ("Congratulations! You won a free iPhone! Call now to claim!", "SPAM"),
    ("Hey, are we still meeting for lunch tomorrow?", "HAM"),
    ("URGENT: Your bank account has been compromised.", "SPAM"),
    ("Don't forget to buy milk on your way home", "HAM"),
    ("FREE entry to win £1000 cash prize. Text WIN now!", "SPAM"),
    ("Meeting rescheduled to 3 PM. See you then.", "HAM")
]

print("📝 Tests de prédiction:")
print("-"*40)

for message, expected in test_messages:
    # Nettoyer
    clean_msg = clean_text(message)
    
    # TF-IDF
    tfidf_features = vectorizer.transform([clean_msg])
    
    # Features numériques
    numeric_features = extract_numeric_features(message)
    numeric_features_sparse = csr_matrix(numeric_features.reshape(1, -1))
    
    # Combiner
    features = hstack([tfidf_features, numeric_features_sparse])
    
    # Prédire
    pred_num = model.predict(features)[0]
    pred_proba = model.predict_proba(features)[0]
    
    prediction = 'SPAM' if pred_num == 1 else 'HAM'
    spam_prob = pred_proba[1]
    
    # Afficher
    print(f"📨 '{message[:50]}...'")
    print(f"   → Prédit: {prediction} ({spam_prob:.2%})")
    print(f"   → Attendu: {expected}")
    print(f"   → {'✅ CORRECT' if prediction == expected else '❌ INCORRECT'}")
    print()

# ============================================
# 11. MISE À JOUR DE L'API
# ============================================

print("\n🌐 11. MISE À JOUR DE L'API")
print("-"*70)

# Lire le fichier API actuel
with open('../api/app.py', 'r', encoding='utf-8') as f:
    api_content = f.read()

# Remplacer les chemins des modèles
api_content = api_content.replace(
    "'../models/logistic_regression_model.joblib'",
    "'../models/logistic_regression_model_final.joblib'"
)
api_content = api_content.replace(
    "'../models/tfidf_vectorizer.joblib'",
    "'../models/tfidf_vectorizer_final.joblib'"
)
api_content = api_content.replace(
    "'../models/label_encoder.joblib'",
    "'../models/label_encoder_final.joblib'"
)

# Sauvegarder
with open('../api/app.py', 'w', encoding='utf-8') as f:
    f.write(api_content)

print("✅ API mise à jour avec le nouveau modèle")

# ============================================
# 12. RÉSUMÉ FINAL
# ============================================

print("\n" + "="*70)
print("🎉 RÉENTRAÎNEMENT TERMINÉ !")
print("="*70)

print(f"\n📊 PERFORMANCES FINALES:")
print(f"   • Accuracy:  {test_accuracy:.2%}")
print(f"   • F1-Score:  {f1:.2%}")

print(f"\n📁 MODÈLES SAUVEGARDÉS:")
print(f"   • logistic_regression_model_final.joblib")
print(f"   • tfidf_vectorizer_final.joblib")
print(f"   • label_encoder_final.joblib")

print(f"\n🚀 POUR TESTER:")
print(f"   1. Redémarre l'API: python api/app.py")
print(f"   2. Teste: python test_api.py")

print(f"\n✅ PROJET COMPLET À 100% !")
# ============================================
# 🔧 CORRECTION DU LABEL ENCODER
# ============================================

print("\n🔧 CORRECTION DU LABEL ENCODER")
print("-"*70)

from sklearn.preprocessing import LabelEncoder

# Créer un vrai LabelEncoder
label_encoder_obj = LabelEncoder()
label_encoder_obj.fit(['ham', 'spam'])

# Sauvegarder
joblib.dump(label_encoder_obj, '../models/label_encoder_final.joblib')
print("✅ Vrai LabelEncoder sauvegardé")

# Mettre à jour l'API
with open('../api/app.py', 'r', encoding='utf-8') as f:
    api_content = f.read()

# Laisser le code tel quel (l'API utilise déjà inverse_transform)
with open('../api/app.py', 'w', encoding='utf-8') as f:
    f.write(api_content)

print("✅ API prête avec le vrai LabelEncoder")

🔥 RÉENTRAÎNEMENT DU MODÈLE LOGISTICREGRESSION

📦 1. CHARGEMENT DES DONNÉES COMPLÈTES
----------------------------------------------------------------------
✅ Données chargées: 5572 messages
   • HAM: 4825 messages
   • SPAM: 747 messages

🧹 2. NETTOYAGE DU TEXTE
----------------------------------------------------------------------
✅ Texte nettoyé

🔤 3. VECTORISATION TF-IDF
----------------------------------------------------------------------
✅ TF-IDF créé: 1000 features

🔢 4. EXTRACTION DES FEATURES NUMÉRIQUES
----------------------------------------------------------------------
✅ Features numériques extraites: 16 features

🔗 5. COMBINAISON DES FEATURES
----------------------------------------------------------------------
✅ Features combinées: 1016 features total
   • TF-IDF: 1000
   • Numériques: 16
   • Cibles: 5572

📊 6. SPLIT DES DONNÉES
----------------------------------------------------------------------
✅ Split terminé:
   • Train: 4457 échantillons
   • Test: 1115 échantil